# SAR-LRA VV_VH Sentinel-1 Image Acquisition

This notebook facilitates the acquisition of Sentinel-1 SAR composite imagery for the VV_VH combination, encompassing both ascending and descending orbits, which can then be utilized to deploy the models.

In [ ]:
from datetime import timedelta
import datetime
import leafmap          # Version 0.22.0
import geemap           # Version 0.24.1
import ee               # Version 0.1.358
from glob import glob
import numpy as np      # Version 1.25.0
import pandas as pd     # Version 2.0.3
import os

# Print versions
print("leafmap:", leafmap.__version__)
print("geemap:", geemap.__version__)
print("Earth Engine API:", ee.__version__)
print("numpy:", np.__version__)
print("pandas:", pd.__version__)

## 2. Autenticate and initialize the Google Earth Engine

In [ ]:
ee.Authenticate()
ee.Initialize()

## 3. Define the Area of Interest

In [ ]:
Map = geemap.Map()
Map

In [ ]:
# Convert the drawn geometry to an Earth Engine Geometry object
geometry = ee.FeatureCollection(Map.draw_features)

## 4. Define temporal buffers and temporal stacks
The following dates are for the Sumatra earthquake on the 25th of February 2022. Change the dates according to the MLE you want to deploy the model for.
IMPORTANT: The landslide MUST be occurred in between the pre_end and post_start dates.

In [ ]:
pre_end = '2022-02-24'
post_start = '2022-02-26'

In [ ]:
### UNCOMMENT THIS TO DEFINE GEOMETRY FOR THE SUMATRA EVENT

# box_coordinates = [[[99.94266597516373, 0.030462907217759412],
#   [100.06812893845004, 0.030462907217759412],
#   [100.06812893845004, 0.14389902329445198],
#   [99.94266597516373, 0.14389902329445198],
#   [99.94266597516373, 0.030462907217759412]]]

# geometry = ee.Geometry.Polygon(box_coordinates)

## 5. Process and Download Sentinel-1 SAR composite images
In certain areas where VH polarization data is unavailable, the notebook may encounter errors as it is designed to download both VV and VH data. The same applies to the ascending and descending orbits.

In [ ]:
print('SENTINEL 1 SAR IMAGE PROCESSING AND ACQUISITION !')
print('_____________________________________________________________________________________')

orbits = ['ASCENDING', 'DESCENDING'] # Orbits to download
pre_days = 60 # pre event stack dimensions 
post_days = 12 # pre event stack dimensions 

# Define date ranges
pre_end = datetime.datetime.strptime(pre_end, "%Y-%m-%d")
pre_start = pre_end - datetime.timedelta(days=pre_days)
print('Pre stack start:', pre_start)
print('Pre stack end:',pre_end)
pre_end = ee.Date(pre_end)
pre_start = ee.Date(pre_start)

post_start = datetime.datetime.strptime(post_start, "%Y-%m-%d")
post_end = post_start + datetime.timedelta(days=post_days)
print('Post stack start:',post_start)
print('Post stack end:',post_end)
post_start = ee.Date(post_start)
post_end = ee.Date(post_end)

for orbit in orbits:
    print('Orbit: ', orbit)
    project_path = ''
    inputs_path = os.path.join(*[project_path, f'deploy/VV_VH/60_{post_days}'])
    outputs_path = os.path.join(*[project_path, 'outputs'])
    print('project_path: ', project_path); print('training_path: ', inputs_path); print('outputs_path: ', outputs_path)
    
    print('_____________________________________________________________________________________')

    # Make Image Collections
    pre_data = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(geometry) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.eq('orbitProperties_pass', orbit)) \
        .filterDate(pre_start, pre_end)

    pre_count = pre_data.size().getInfo()
    print('Pre image count:', pre_count)

    # Get a list of images in the collection
    pre_list = pre_data.toList(pre_count)

    # Iterate through each image in the collection
    for i in range(pre_count):
        image = ee.Image(pre_list.get(i))
        image_id = image.id().getInfo()
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        print('Image ID:', image_id, ' Date:', date)

    post_data = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(geometry) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.eq('orbitProperties_pass', orbit)) \
        .filterDate(post_start, post_end)

    post_count = post_data.size().getInfo()
    print('Post image count:', post_count)

    # Get a list of images in the collection
    post_list = post_data.toList(post_count)

    # Iterate through each image in the collection
    for i in range(post_count):
        image = ee.Image(post_list.get(i))
        image_id = image.id().getInfo()
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        print('Image ID:', image_id, ' Date:', date)

    ### VV ###
    pre_corrected_masked_VV = pre_data.select('VV') # Select VV polarization
    pre_median_VV = pre_corrected_masked_VV.reduce(ee.Reducer.median())  # Calculate median
    pre_median_VV = pre_median_VV.select(0).rename('preVV')  # Rename bands


    post_corrected_masked_VV = post_data.select('VV')  # Select VV polarization
    post_median_VV = post_corrected_masked_VV.reduce(ee.Reducer.median())  # Calculate median
    post_median_VV = post_median_VV.select(0).rename('postVV')  # Rename bands

    diff_VV = post_median_VV.subtract(pre_median_VV).rename('diffVV') # Change detection (post event - pre event)

     ### VH ###
    pre_corrected_masked_VH = pre_data.select('VH')   # Select VV polarization
    pre_median_VH = pre_corrected_masked_VH.reduce(ee.Reducer.median())  # Calculate median
    pre_median_VH = pre_median_VH.select(0).rename('preVH')  # Rename bands


    post_corrected_masked_VH = post_data.select('VH')  # Select VV polarization
    post_median_VH = post_corrected_masked_VH.reduce(ee.Reducer.median())  # Calculate median
    post_median_VH = post_median_VH.select(0).rename('postVH')  # Rename bands

    diff_VH = post_median_VH.subtract(pre_median_VH).rename('diffVH') # Change detection (post event - pre event)

    DS = post_median_VV.addBands(post_median_VH).addBands(diff_VV).addBands(diff_VH) # Create composite SAR image
    DS = DS.clip(geometry) # Clip composite SAR image to Area of Interest
    
    ### GEE does not allow big downloads - therefore we dowload grid by grid, and re-merge afterwards.

    # create fishnet (sample grid)
    fishnet = geemap.fishnet(geometry, h_interval=0.5, v_interval=0.5, delta=1)

    # download composite tiles
    geemap.download_ee_image_tiles(DS, fishnet, inputs_path+'/'+'/VV_VH_'+orbit+'/', prefix="VV_VH_"+orbit+'_'+'_', scale=10, crs=ee.Projection('EPSG:4326'))

    # merge tiles and save
    leafmap.merge_rasters(inputs_path+'/'+'/VV_VH_'+orbit, output=inputs_path+'/'+'/SAR_'+orbit+'_'+'.tif', input_pattern='*.tif')

    # delete useless rasters inside folders
    s2s = glob(inputs_path+'/'+'/VV_VH_'+orbit+'/*.tif')

    for s2_file in s2s:
        os.remove(s2_file)
    os.rmdir(inputs_path+'/'+'/VV_VH_'+orbit)

print('DONE !!!')

### 6. Display the last downloaded Sentinel-1 Composite Image


In [ ]:
Map.close()
Map = geemap.Map()
Map.addLayer(DS, {'bands': ['postVV'], 'min': [-28], 'max': [4], 'gamma': 0.65}, 'postVV', False)
Map.addLayer(DS, {'bands': ['postVH'], 'min': [-28], 'max': [4], 'gamma': 0.65}, 'postVH', False)
Map.addLayer(DS, {'bands': ['diffVV'], 'min': [-28], 'max': [4], 'gamma': 0.65}, 'diffVV', True)
Map.addLayer(DS, {'bands': ['diffVH'], 'min': [-28], 'max': [4], 'gamma': 0.65}, 'diffVH', False)

Map.centerObject(geometry)

Map